In [1]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n"

In [80]:
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,df_cost,df_esti,rel, df_bcaw]:
    colname_modi(i)

In [81]:
# pre screen data
df_cost = df_cost.dropna(axis=1, how = 'all').fillna(0)
frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)
df_cost = df_cost.loc[:, frame[frame[0] == False].index.tolist()]
del frame
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/1032199403.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)


In [73]:
def data_prepare():
    """
    data prepare
    """
    df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

opportunity = data_prepare()
# del contract_summary

In [17]:
df_list = [opportunity,df_bill,df_cost,df_esti]


In [18]:
def check_ouname(df):
    print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

for i in df_list:
    check_ouname(i)

del df_list,i

Empty DataFrame
Columns: [0]
Index: [] 	
              0
13  cost_center 	
         0
1  ou_code 	
              0
12  cost_center 	


In [168]:
def data_clean(df, substr_year, **kwargs):
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4)
    df = df.groupby(['account_name_en','account_name_cn', substr_year]).agg({
        **kwargs
    })
    return df 
def to_cust_level(df,  substr_year, **kwargs):
    """
    merge all to cust level
    """

    try:
        df1 = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
        # print(df1.head(2))
        df1 = data_clean(df1, substr_year,  **kwargs)
    except:
        df1 = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
        # print(substr_year, **kwargs)
        df1 = data_clean(df1, substr_year,  **kwargs) 
    return df1

In [169]:
to_cust_level(df = df_bill, substr_year = 'bms_bill_end_date' ,\
    bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/3760012408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


bms_amount  \
                                                               sum   
account_name_en    account_name_cn bms_bill_end_date                 
ASML               阿斯麦             2020               2.986967e+07   
                                   2021               3.929254e+07   
Abbott Nutrition   雅培 营养品          2020               1.931013e+07   
                                   2021               2.080320e+07   
Accuray            安科锐             2020               2.022754e+05   
...                                                            ...   
Yudo Motor         云度汽车            2021               6.146397e+05   
ZEBRA TECHNOLOGIES 斑马技术            2020               1.099901e+06   
                                   2021               7.185659e+10   
ZTE                中兴通讯            2020               2.104321e+06   
                                   2021               4.407333e+06   

                                                                                           bms_fee_type  \
                                                                                                    set   
account_name_en    account_name_cn bms_bill_end_date                                                      
ASML               阿斯麦             2020               {security_equipment_fee, office_facility_fee, ...   
                                   2021               {customs_agent_fee_opening_fee, other_short_ba...   
Abbott Nutrition   雅培 营养品          2020               {machinery_equipment_fee, packing_fee, overtim...   
                                   2021               {forklift_fee, machinery_equipment_fee, packin...   
Accuray            安科锐             2020               {packing_fee, overtime_pay_fee, other_place_fe...   
...                                                                                                 ...   
Yudo Motor         云度汽车            2021               {short_barge_transport_fee, wage_fee, short_ba...   
ZEBRA TECHNOLOGIES 斑马技术            2020               {machinery_equipment_fee, packing_fee, wage_fe...   
                                   2021               {it_device_fee, goods_shelve_fee, forklift_fee...   
ZTE                中兴通讯            2020               {short_barge_transport_fee, wage_fee, overtime...   
                                   2021               {short_barge_transport_fee, wage_fee, overtime...   

                                                              \
                                                     nunique   
account_name_en    account_name_cn bms_bill_end_date           
ASML               阿斯麦             2020                   18   
                                   2021                   29   
Abbott Nutrition   雅培 营养品          2020                    9   
                                   2021                   12   
Accuray            安科锐             2020                    7   
...                                                      ...   
Yudo Motor         云度汽车            2021                    3   
ZEBRA TECHNOLOGIES 斑马技术            2020                    8   
                                   2021                   12   
ZTE                中兴通讯            2020                    6   
                                   2021                    7   

                                                                                           bms_fee_name  \
                                                                                                    set   
account_name_en    account_name_cn bms_bill_end_date                                                      
ASML               阿斯麦             2020               {其他报关费, 代理费, 空调设备, 查验费, 仓储费, 其他增值服务费, 办公设施费, 安...   
                                   2021               {空调设备, 安防设备, 查验费, 能源费, 特殊报关代理费, 短驳运输费用开口收费, 叉车...   
Abbott Nutrition   雅培 营养品          2020               {其他增值服务费, 仓储费, 包装费, 加班费, 场内操作费, 贴标费, 机器设备, 其他设...   
                                   20

In [154]:
def tst(substr_year,**kwargs):
    df_bill[substr_year] = df_bill[substr_year].astype(str).str.slice(0,4)
    return  df_bill.groupby([substr_year, 'cost_center']).agg({
        **kwargs
    }).reset_index()


# def get_options(list_stocks):
#     dict_list = []
#     for i in list_stocks:
#         dict_list.append({'label': i, 'value': i})
#     return dict_list

In [139]:
data_clean(df = df_bill, substr_year = 'bms_bill_end_date' ,\
    bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

KeyError: 'account_name_en'

In [155]:
tst(substr_year = 'bms_bill_end_date' ,\
    bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

bms_bill_end_date          cost_center    bms_amount  \
                                                     sum   
0                2019          Dsc_Test_06  4.772606e+04   
1                2019           FORDXCQFDS  1.514162e+06   
2                2019      dsc-test-01-cus  6.212195e+03   
3                2020           ABBOTBJXWS  1.350914e+06   
4                2020           ABBOTCDXXS  8.875027e+05   
..                ...                  ...           ...   
576              2021  fcy-test-pro_testQA  7.592461e+00   
577              2021                 xsqd  5.725197e+03   
578              2021          xsqd_testQA  0.000000e+00   
579              2022           EVISUSHDQR  8.778146e+05   
580              2022           SGMWALZH1S -1.027361e+04   

                                          bms_fee_type          \
                                                   set nunique   
0    {wage_fee, overtime_pay_fee, electricity_fee, ...       6   
1                                {other_operating_fee}       1   
2    {wage_fee, electricity_fee, overtime_pay_fee, ...       6   
3    {on_site_operation_fee, overtime_pay_fee, othe...       5   
4    {on_site_operation_fee, overtime_pay_fee, othe...       5   
..                                                 ...     ...   
576  {other_short_barge_transport_fee, claim_and_re...       2   
577  {customs_agent_fee_opening_fee, rebate_fee, ov...       8   
578                                              {nan}       0   
579  {short_barge_transport_fee, short_barge_fee_op...       2   
580          {overtime_pay_fee, on_site_operation_fee}       2   

                                          bms_fee_name bms_bill_id  
                                                   set     nunique  
0                        {出库费, 贴标费用, 电费, 加班费, 工资, 入库费}           1  
1                                              {其他操作费}           1  
2                        {出库费, 贴标费用, 电费, 加班费, 工资, 入库费}           4  
3                    {仓储费, 其他增值服务费, 加班费, 场内操作费, 其他设备费}           9  
4                    {仓储费, 其他增值服务费, 加班费, 场内操作费, 其他设备费}           8  
..                                                 ...         ...  
576                             {索赔及返利费用开口收费, 其他短驳运输费}           3  
577  {报关代理费用开口收费, 能源费, 返利费, 加班费, 场内操作费, 入库费, 短驳运输费用...           3  
578                                              {nan}           2  
579                                {短驳运输费用开口收费, 短驳运输费}           1  
580                                       {加班费, 场内操作费}           1  

[581 rows x 7 columns]

In [ ]:
df_sub[df_sub['account_name_en'] == 'ABC']['annual_average_revenue']

In [ ]:
min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

